# Tratamento inicial dos dados

Objetivo:
1. Carregar os dados;
2. Selecionar as colunas que serão lidas;
3. Fazer uma amostragem aleatória de 800.000 dados (~20% do dataset);
4. Salvar em um novo ``.csv`` que seja mais leve

## Seleção de variáveis e amostragem dos dados

**NOTA SOBRE O RESULTADO DA LLM:** O algoritmo gerado para selecionar e amostrar os dados foi bastante satisfatório, a não ser por um pequeno erro lógico no final: na fase ``# junta tudo``, o código tenta tirar novamente 800_000 amostras de uma lista que já tem (teoricamente) 800_000 amostras, então essa amostragem não faz sentido. Na prática, ainda, o código provavelmente vai gerar um erro, já que nosso banco não tem exatamente 4M registros e o resultado terá um pouco menos que 800k amostras.

In [ ]:
""""
GERA O ARQUIVO SAMPLE, NÃO RODAR SE O ARQUIVO JÁ ESTIVER CRIADO
"""

import pandas as pd
import numpy as np
from pathlib import Path

# Configurações
DATA_PATH = Path().resolve() / 'data'
ARQUIVO_ORIGEM_PATH = DATA_PATH / 'raw' / 'microdados_enem_2023.csv'
ARQUIVO_DESTINO_PATH = DATA_PATH / 'raw' / 'microdados_enem_2023_sample.csv'

colunas_desejadas = [
    'NU_INSCRICAO', 'TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
    'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO',
    'IN_TREINEIRO', 'CO_MUNICIPIO_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC',
    'TP_SIT_FUNC_ESC', 'CO_MUNICIPIO_PROVA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH',
    'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
    'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'TP_LINGUA',
    'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
    'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q006', 'Q025'
]

# Parâmetros
tamanho_amostra_final = 800_000
tamanho_arquivo_total = 4_000_000  # Aproximadamente
frac_amostragem = tamanho_amostra_final / tamanho_arquivo_total

# Leitura em chunks
chunk_size = 500_000
amostras = []

for chunk in pd.read_csv(ARQUIVO_ORIGEM_PATH, usecols=colunas_desejadas, chunksize=chunk_size, sep=';', encoding='latin1'):
    amostra_chunk = chunk.sample(frac=frac_amostragem, random_state=553)
    amostras.append(amostra_chunk)

# Junta tudo
# amostra_final = pd.concat(amostras).sample(n=tamanho_amostra_final, random_state=42) # resultado da LLM
amostra_final = pd.concat(amostras)

# Salva no CSV
amostra_final.to_csv(ARQUIVO_DESTINO_PATH, index=False)

print(f"Amostra salva em {ARQUIVO_DESTINO_PATH}")

## Leitura eficiente do dataframe amostrado

**NOTA SOBRE O RESULTADO DA LLM:** Constroi um dicionário de dtypes de forma inteligente, mas: i) faz um ``col for col ...`` desnecessariamente em ``colunas_float``. 

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# Configurações
DATA_PATH = Path().resolve() / 'data'
ARQUIVO_AMOSTRA_PATH = DATA_PATH / 'raw' / 'microdados_enem_2023_sample.csv'

# Definição dos tipos
colunas_float = [
    'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
    'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
    'NU_NOTA_COMP5', 'NU_NOTA_REDACAO'
]

colunas_string = [
    'NU_INSCRICAO', 'CO_MUNICIPIO_ESC', 'CO_MUNICIPIO_PROVA'
]

# Captura os nomes das colunas
colunas = pd.read_csv(ARQUIVO_AMOSTRA_PATH, nrows=0, encoding='latin1').columns.tolist()

# Preparar o dicionário de tipos
dtypes = {}
for col in colunas:
    if col in colunas_float:
        dtypes[col] = 'float32'
    elif col in colunas_string:
        dtypes[col] = 'string'
    else:
        dtypes[col] = 'category'

# Leitura com tipos otimizados
df = pd.read_csv(ARQUIVO_AMOSTRA_PATH, dtype=dtypes, encoding='latin1')

# Informações para conferência
print("="*40)
print(f"Memória usada: {df.memory_usage(deep=True).sum() / (1024 ** 2):.2f} MB")
print(f"Quantidade de colunas: {df.shape[1]}")
print(f"Quantidade de linhas: {df.shape[0]}")
print("="*40)

# Tipos de dados
print("\nTipos de dados por coluna:")
print(df.dtypes)

## Exploratory Data Analysis

In [ ]:
# Checagem de valores nulos
print("\nAno que concluiu:")
nulos = df.isnull().sum()
percentual_nulos = (nulos / len(df)) * 100


tp_escola = df['TP_ANO_CONCLUIU'].value_counts(dropna=False)
tp_escola_pct = df['TP_ANO_CONCLUIU'].value_counts(normalize=True, dropna=False) * 100

resultado = pd.DataFrame({
    'Frequência': tp_escola,
    'Percentual (%)': tp_escola_pct.round(2)
})

print(resultado)

In [ ]:
# Checagem de valores nulos
print("\nConclusão:")
nulos = df.isnull().sum()
percentual_nulos = (nulos / len(df)) * 100


tp_escola = df['TP_ST_CONCLUSAO'].value_counts(dropna=False)
tp_escola_pct = df['TP_ST_CONCLUSAO'].value_counts(normalize=True, dropna=False) * 100

resultado = pd.DataFrame({
    'Frequência': tp_escola,
    'Percentual (%)': tp_escola_pct.round(2)
})

print(resultado)

In [ ]:

filtro = df[df['TP_ANO_CONCLUIU'] == '0']
grupo = filtro.groupby('TP_ST_CONCLUSAO', dropna=False)['NU_INSCRICAO']

resultado = pd.DataFrame({
    'Total': grupo.count()
})

resultado['Percentual (%)'] = (resultado['Total'] / resultado['Total'].sum() * 100).round(2)

print(resultado)
print(resultado['Total'].sum())

In [ ]:
df[df['TP_ST_CONCLUSAO'] != '1'].count()

In [ ]:
tirou0 = df.NU_NOTA_CH[df.NU_NOTA_CH == 0].count()
tirou_mais_q_0 = df.NU_NOTA_CH[df.NU_NOTA_CH > 0].count()
total = df.NU_NOTA_CH.count()

print(tirou0, '\t', 100.0*tirou0/total)
print(tirou_mais_q_0, '\t', 100.0*tirou_mais_q_0/total)
print(total, '\t', 100.0*total/total)

In [ ]:
df[df.NU_NOTA_CH == 0].groupby('TP_PRESENCA_CH').count()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Tabela cruzada entre duas colunas categóricas
tabela = pd.crosstab(df['TP_ANO_CONCLUIU'], df['TP_ST_CONCLUSAO'][df['TP_ST_CONCLUSAO'] == 0])

# Heatmap
sns.heatmap(tabela, annot=True, fmt='d', cmap='Blues')
plt.xlabel('COLUNA2')
plt.ylabel('COLUNA1')
plt.title('Heatmap entre COLUNA1 e COLUNA2')
plt.show()

In [ ]:
# Checagem de valores nulos
print("\nValores nulos por coluna:")
nulos = df.isnull().sum()
percentual_nulos = (nulos / len(df)) * 100

# Criando um dataframe com os valores absolutos e percentuais
df_nulos = pd.DataFrame({
    'Valores Nulos': nulos,
    'Percentual Nulos (%)': percentual_nulos
})

# Exibindo o dataframe de nulos
print(df_nulos)

In [ ]:
# Checagem de valores nulos
print("\nTipo de escola:")

tp_escola = df['TP_ESCOLA'].value_counts(dropna=False)
tp_escola_pct = df['TP_ESCOLA'].value_counts(normalize=True, dropna=False) * 100

resultado = pd.DataFrame({
    'Frequência': tp_escola,
    'Percentual (%)': tp_escola_pct.round(2)
})

print(resultado)

In [ ]:
median = df.NU_NOTA_MT.median() 
acima_da_mediana = df.NU_NOTA_MT[df.NU_NOTA_MT > median]

df.groupby().agg({'NU_NOTA_MT': 'mean', 'SEXO': 'count'})

In [ ]:
# Checagem de valores nulos
print("\nTipo de ensino:")

tp_escola = df['TP_ENSINO'].value_counts(dropna=False)
tp_escola_pct = df['TP_ENSINO'].value_counts(normalize=True, dropna=False) * 100

resultado = pd.DataFrame({
    'Frequência': tp_escola,
    'Percentual (%)': tp_escola_pct.round(2)
})

print(resultado)

In [ ]:
# Checagem de valores nulos
print("\nPresença Mat:")

tp_escola = df['TP_PRESENCA_MT'].value_counts(dropna=False)
tp_escola_pct = df['TP_PRESENCA_MT'].value_counts(normalize=True, dropna=False) * 100

resultado = pd.DataFrame({
    'Frequência': tp_escola,
    'Percentual (%)': tp_escola_pct.round(2)
})

print(resultado)

In [ ]:
# Checagem de valores nulos
print("\n:")

tp_escola = df['TP_PRESENCA_MT'].value_counts(dropna=False)
tp_escola_pct = df['TP_PRESENCA_MT'].value_counts(normalize=True, dropna=False) * 100

resultado = pd.DataFrame({
    'Frequência': tp_escola,
    'Percentual (%)': tp_escola_pct.round(2)
})

print(resultado)


Abaixo, segue a análise descritiva da distribuição das notas por área de conhecimento. Foram plotados 5 histogramas, um para cada área de conhecimento. Como o histograma da redação tinha uma distribuição muito diferente dos demais, foi decidido plotá-lo separadamente dos outros.

Percebe-se que a mediana está muito próxima da média, sugerindo uma distribuição simétrica. Destaca-se a presença de notas zeradas em todas as áreas do conhecimento. Esse comportamento é curioso e será objeto de investigação.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
colunas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT']
titulos = ['Nota da prova de Ciências da Natureza',
           'Nota da prova de Ciência Humanas',
           'Nota da prova de Linguagens e Códigos',
           'Nota da prova de Matemática']

for ax, col, tit in zip(axes.flatten(), colunas, titulos):
    sns.histplot(df[col], bins='auto', ax=ax)

    media = df[col].mean()
    std = df[col].std()
    mediana = df[col].median()

    ax.axvline(media, color='red', linestyle='--', linewidth=1.0, label='Média')
    ax.axvline(mediana, color='blue', linestyle='--', linewidth=1.0, label='Mediana')
    ax.axvline(media + std, color='orange', linestyle='--', linewidth=0.5, label='+1 desvio')
    ax.axvline(media - std, color='orange', linestyle='--', linewidth=0.5, label='-1 desvio')
    ax.axvline(media + 2*std, color='green', linestyle='--', linewidth=0.5, label='+2 desvios')
    ax.axvline(media - 2*std, color='green', linestyle='--', linewidth=0.5, label='-2 desvios')

    ax.set_title(f'Histograma \n{tit}', fontsize=12)
    ax.set_xlabel('Nota')
    ax.set_ylabel('')
    ax.set_xlim(-50, 1000)
    ax.set_ylim(0, 8000)

    if col == colunas[0]:
        ax.legend()

plt.tight_layout()
plt.show()


# Gráfico da Redação separado
col = 'NU_NOTA_REDACAO'
media = df[col].mean()
std = df[col].std()
mediana = df[col].median()

plt.figure(figsize=(6, 4))
sns.histplot(df[col], bins=50)

plt.axvline(media, color='red', linestyle='--', linewidth=1.0, label='Média')
plt.axvline(mediana, color='blue', linestyle='--', linewidth=1.0, label='Mediana')
plt.axvline(media + std, color='orange', linestyle='--', linewidth=0.5, label='+1 desvio')
plt.axvline(media - std, color='orange', linestyle='--', linewidth=0.5, label='-1 desvio')
plt.axvline(media + 2*std, color='green', linestyle='--', linewidth=0.5, label='+2 desvios')
plt.axvline(media - 2*std, color='green', linestyle='--', linewidth=0.5, label='-2 desvios')

plt.title('Histograma \nNota da prova de redação', fontsize=12)
plt.xlabel('Nota')
plt.ylabel('')
plt.xlim(-50, 1000)
plt.tight_layout()
plt.show()


Abaixo, segue a análise descritiva da distribuição das notas por competência na prova de redação. Foram plotados 5 histogramas, um para cada competência.

Percebe-se que a mediana está muito próxima da média, sugerindo uma distribuição simétrica. Destaca-se a presença de notas zeradas em todas as competências, como foi observado também para as notas das provas por área de conhecimento acima. Esse comportamento é curioso e será objeto de investigação também. ???

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(12, 12))
colunas = ['NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5']
titulos = ['Nota Competência 1 - Redação',
           'Nota Competência 2 - Redação',
           'Nota Competência 3 - Redação',
           'Nota Competência 4 - Redação',
           'Nota Competência 5 - Redação']

for ax, col, tit in zip(axes.flatten(), colunas, titulos):
    sns.histplot(df[col], bins=10, ax=ax)

    media = df[col].mean()
    std = df[col].std()
    mediana = df[col].median()

    ax.axvline(media, color='red', linestyle='--', linewidth=1.0, label='Média')
    ax.axvline(mediana, color='blue', linestyle='--', linewidth=1.0, label='Mediana')
    ax.axvline(media + std, color='orange', linestyle='--', linewidth=0.5, label='+1 desvio')
    ax.axvline(media - std, color='orange', linestyle='--', linewidth=0.5, label='-1 desvio')
    ax.axvline(media + 2*std, color='green', linestyle='--', linewidth=0.5, label='+2 desvios')
    ax.axvline(media - 2*std, color='green', linestyle='--', linewidth=0.5, label='-2 desvios')

    ax.set_title(f'Histograma \n{tit}', fontsize=12)
    ax.set_xlabel('Nota')
    ax.set_ylabel('')
    ax.set_xlim(0, 200) 
    ax.set_ylim(0, 250000)
  
    if col == colunas[0]:
        ax.legend()

# Remove o sexto subplot vazio (última célula da grade 2x3)
if len(colunas) < len(axes.flatten()):
    for ax in axes.flatten()[len(colunas):]:
        fig.delaxes(ax)

plt.tight_layout()
plt.show()

Com intuito de categorizar as variáveis numéricas notas (área de conhecimento e competencia da prova de redação) foram adotadas 5 faixas: ausente, zerou, baixa, media e alta. O critério de classificação das notas em cada uma das faixas foi:  

1. ausente: valores NA do banco, que significa pessoas que não compareceram ou tiveram sua prova anulada no dia da prova da respectiva área de conhecimento.
2. zerou: valor da nota igual a 0 (zero).
3. baixa: valor da nota inferior a media - 1 desvio
4. media: valor da nota entre a media - 1 desvio e media + 1 desvio
5. alta: valor da nota superior a media + 1 desvio

In [ ]:
# Transformacao das variáveis numéricas em categóricas, segunda as faixas estipuladas

colunas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 
           'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5']

for col in colunas:
    media = df[col].mean()
    std = df[col].std()

    condicoes = [
        df[col].isna(),
        df[col] == 0,
        df[col] < media - std,
        df[col] <= media + std,
        df[col] > media + std
    ]

    categorias = ['ausente', 'zerou', 'baixa', 'media', 'alta']

    resultado = np.select(condicoes, categorias, default='erro')

    df[col] = pd.Categorical(
        resultado,
        categories=['ausente', 'zerou', 'baixa', 'media', 'alta'],
        ordered=True
    )

In [ ]:
# Proporcao das faixas das notas

print("\n:")

colunas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO',
           'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5']
contagens = {}
percentual = {}

for col in colunas:
    
    contagens[f'tp_{col}'] = df[col].value_counts(dropna=False)
    percentual[f'tp_{col}_pct'] = df[col].value_counts(normalize=True, dropna=False) * 100

    resultado = pd.DataFrame({
        'Frequência': contagens[f'tp_{col}'],
        'Percentual (%)': percentual[f'tp_{col}_pct'].round(2)
    })
    
    print(resultado, '\n')